In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import Model

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))
from deepexplain.tensorflow import DeepExplain
from keras import backend as K

Using TensorFlow backend.


In [2]:
#toy model with based on Integrated Gradients paper (https://arxiv.org/pdf/1703.01365.pdf)
with DeepExplain(session=K.get_session()) as de:
    model_2 = Sequential()
    model_2.add(Dense(2, activation=Activation('relu'), input_dim=2))
    model_2.add(Dense(1, activation=Activation('relu')))
    
    w1 = np.array([[1, 0], [0, 1]])
    b1 = np.array([-1, 0])
    model_2.layers[0].set_weights([w1, b1]) #x1-1, x2
    
    w2 = np.array([[1], [-1]])
    b2 = np.array([0])
    model_2.layers[1].set_weights([w2, b2]) #z1 - 1 - z2 
    #print(model.layers[0].get_weights())
    
    #alpha = .1
    #print(-15.75*alpha+9)
    #print(K.get_session().run(model.layers[-1].output, feed_dict={model.layers[0].input:np.array([[-2*alpha, 
                                         #                                                 3*alpha]])}))    
    baseline = np.array([[0,0]])
    xs = np.array([[3, 1]])
    input_tensor = model_2.layers[0].input
    target_tensor = model_2.layers[-1].output
    #fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    #target_tensor = fModel(input_tensor)
    print(input_tensor.shape)
    print(target_tensor.shape)
    print(model_2.predict(baseline))
    print(model_2.predict(xs)) #sanity check -
    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(?, 2)
(?, 1)

[[0.]]
[[1.]]


/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [3]:
ground_truth = np.array([1.5, -0.5]) #self-calculated
steps = [5]#1, 5, 10, 20, 50, 100, 1000]# 200, 300, 400, 500]
methods = ['idl_true']
attribs_2 = {}
diffs_2 = {}
xs = np.array([[3, 1]])

with DeepExplain(session=K.get_session()) as de:
    #attrib_deeplift = de.explain('deeplift', target_tensor, input_tensor, xs)[0] #1 numpy array

    for method in methods:
        attribs_2[method] = []
        diffs_2[method] = []
        for step in steps:
            attrib = de.explain(method, target_tensor, input_tensor, xs, steps=step)[0] #1 numpy array
            attribs_2[method].append(attrib) 
            diffs_2[method].append(np.sum(np.abs(attrib - ground_truth))) #append 1 number
        
    #attribs['intgrad'] = de.explain('intgrad', target_tensor, input_tensor, xs, steps=steps)
    #attribs['integdeeplift'] = de.explain('integdeeplift', target_tensor, input_tensor, xs, steps=steps)
    print("Done!")

print(attribs_2)
#print(attrib_deeplift)

[array([[-1.,  0.]], dtype=float32), array([[0.]], dtype=float32)]
OPS
dense_1/activation_1/Relu
[[-1.  0.]]
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


dense_2/activation_2/Relu
[[0.]]
DeepLIFT: references ready
{'dense_1/activation_1/Relu': <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32_ref>, 'dense_2/activation_2/Relu': <tf.Variable 'Variable_1:0' shape=(1, 1) dtype=float32_ref>}
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


GRAD
output Tensor("dense_2/activation_2/Relu:0", shape=(?, 1), dtype=float32)
input Tensor("dense_2/BiasAdd:0", shape=(?, 1), dtype=float32)
ref_input <tf.Variable 'Variable_1:0' shape=(1, 1) dtype=float32_ref>
ref_output Tensor("gradients/dense_2/activation_2/Relu_grad/Relu:0", shape=(1, 1), dtype=float32)
delta_out Tensor("gradients/dense_2/activation_2/Relu_grad/sub:0", shape=(?, 1), dtype=float32)
delta_in Tensor("gradients/dense_2/activation_2/Relu_grad/sub_1:0", shape=(?, 1), dtype=float32)
result Tensor("gradients/dense_2/activation_2/Relu_grad/Select:0", shape=(?, 1), dtype=float32)
GRAD
output Tensor("dense_1/activation_1/Relu:0", shape=(?, 2), dtype=float32)
input Tensor("dense_1/BiasAdd:0", shape=(?, 2), dtype=float32)
ref_input <tf.Variable 'Variable:0' shape=(1, 2) dtype=float32_ref>
ref_output Tensor("gradients/dense_1/activation_1/Relu_grad/Relu:0", shape=(1, 2), dtype=float32)
delta_out Tensor("gradients/dense_1/activation_1/Relu_grad/sub:0", shape=(?, 2), dtype=float3